In [27]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import xgboost as xgb

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

In [28]:
dfCOESBase = pd.read_excel('DemandaCOES.xlsx', skiprows=3)
dfCOESBase = dfCOESBase[['FECHA','EJECUTADO']]
dfCOESBase['FECHA'] = pd.to_datetime(dfCOESBase['FECHA'], format='%d/%m/%Y %H:%M')

In [29]:
dfCOESBase

,FECHA,EJECUTADO
0,2020-01-01 00:30:00,5588.64463
1,2020-01-01 01:00:00,5549.41169
2,2020-01-01 01:30:00,5461.26767
3,2020-01-01 02:00:00,5378.65177
4,2020-01-01 02:30:00,5277.15003
...,...,...
74539,2024-04-01 22:00:00,NaN
74540,2024-04-01 22:30:00,NaN
74541,2024-04-01 23:00:00,NaN
74542,2024-04-01 23:30:00,NaN


In [30]:


# devolvemos todo 30 minutos para que asi no tengamos fechas del dia siguiente.
# antes del cambio  las 14:00 implicaban datos desde las 13:30 hasta las 14.  Luego del cambio significará desde las 14 hasta las 14:30
dfCOESBase['FECHA']= dfCOESBase['FECHA'] - pd.Timedelta(minutes=30)

# Creamos las nuevas columnas
dfCOESBase['Mes'] = dfCOESBase['FECHA'].dt.month
dfCOESBase['ano'] = dfCOESBase['FECHA'].dt.year
dfCOESBase['fechaYYYYMMDD'] = dfCOESBase['FECHA'].dt.strftime('%Y-%m-%d')
dfCOESBase['fechaYYYYMMDD'] = pd.to_datetime(dfCOESBase['fechaYYYYMMDD'])
dfCOESBase['real'] = dfCOESBase['EJECUTADO']

In [31]:

# la NOCHE agrupará desde las 5pm hasta las 12pm
dfCOESNoche = dfCOESBase[(dfCOESBase['FECHA'].dt.hour >= 17) & (dfCOESBase['FECHA'].dt.hour < 23)].copy()
dfCOESNoche = dfCOESNoche.groupby('fechaYYYYMMDD')['EJECUTADO'].max().reset_index()
dfCOESNoche.rename(columns={'EJECUTADO': 'Noche'}, inplace=True)

In [32]:


# DIA: se toma solo el máximo de las madrugada hasta las 2pm (incluida)
dfCOESDia = dfCOESBase[(dfCOESBase['FECHA'].dt.hour >= 0) & (dfCOESBase['FECHA'].dt.hour < 14)].reset_index().copy()
dfCOESDia= dfCOESDia.groupby('fechaYYYYMMDD')['EJECUTADO'].max().reset_index()
dfCOESDia.rename(columns={'EJECUTADO': 'Dia'}, inplace=True)

In [33]:

# TARDE: se toma desde las 2pm hasta las 5pm
dfCOESTarde = dfCOESBase[(dfCOESBase['FECHA'].dt.hour >= 14) & (dfCOESBase['FECHA'].dt.hour < 17)].reset_index().copy()
dfCOESTarde= dfCOESTarde.groupby('fechaYYYYMMDD')['EJECUTADO'].max().reset_index()
dfCOESTarde.rename(columns={'EJECUTADO': 'Tarde'}, inplace=True)

In [34]:

dfCOESTotal = pd.merge(dfCOESDia, dfCOESTarde, on='fechaYYYYMMDD', how='left')

dfCOESTotal = pd.merge(dfCOESTotal, dfCOESNoche, on='fechaYYYYMMDD', how='left')

# OJO, QUITAMOS LOS DATOS ANTERIORES AL 2021 PUES SON MUY ATIPICOS
dfCOESTotal = dfCOESTotal[dfCOESTotal['fechaYYYYMMDD'] >= '2021-01-01']

dfCOESTotal['ano'] = dfCOESTotal['fechaYYYYMMDD'].dt.year
dfCOESTotal['mes'] = dfCOESTotal['fechaYYYYMMDD'].dt.month

dfCOESTotal = dfCOESTotal.reset_index(drop=True)

In [35]:
dfCOESTotal[dfCOESTotal['fechaYYYYMMDD']>= '2024-03-01']

,fechaYYYYMMDD,Dia,Tarde,Noche,ano,mes
1155,2024-03-01,7610.47152,7599.87572,7242.94498,2024,3
1156,2024-03-02,7428.74069,7269.19111,7375.16264,2024,3
1157,2024-03-03,6684.95852,6602.11623,7268.10306,2024,3
1158,2024-03-04,7927.10175,7940.07334,7640.77741,2024,3
1159,2024-03-05,7742.17210,7766.70833,7492.23652,2024,3
1160,2024-03-06,7744.91547,7823.83213,7458.40400,2024,3
1161,2024-03-07,7851.91114,7891.94135,7447.04639,2024,3
1162,2024-03-08,7711.78231,7629.06256,7488.32811,2024,3
1163,2024-03-09,7581.47274,7390.91123,7376.95169,2024,3
1164,2024-03-10,6732.26281,6652.27203,7282.17310,2024,3


In [36]:
diaAPredecir = dfCOESTotal['fechaYYYYMMDD'].max()
diaAPredecir

Timestamp('2024-04-01 00:00:00')

In [37]:
dfCOESTotal.loc[dfCOESTotal['fechaYYYYMMDD'] == diaAPredecir, 'Noche'] = np.nan
dfCOESTotal.tail()

,fechaYYYYMMDD,Dia,Tarde,Noche,ano,mes
1182,2024-03-28,7102.78503,6961.58835,7384.08005,2024,3
1183,2024-03-29,6682.70887,6485.00370,7138.48225,2024,3
1184,2024-03-30,7200.17800,6935.58222,7262.40741,2024,3
1185,2024-03-31,6576.77453,6524.72391,7223.41316,2024,3
1186,2024-04-01,7866.30184,7856.88739,NaN,2024,4
